In [1]:
import numpy as np
import copy
from time import perf_counter
from collections import Counter, deque
from functools import lru_cache
#import numba
from itertools import permutations, combinations, product

In [2]:
f = open("input.txt", "r")
rawstring = f.read()
f.close()

In [44]:
class Field():
    def __init__(self, map) -> None:
        self.map = map
        self.homes = {
            'A': [[2, 3], [3, 3]],
            'B': [[2, 5], [3, 5]],
            'C': [[2, 7], [3, 7]],
            'D': [[2, 9], [3, 9]]
        }
        self.taboos = [[1, 3], [1, 5], [1, 7], [1, 9]]
        self.movable_pos = [[1, 1], [1, 2], [1, 4],
                            [1, 6], [1, 8], [1, 10], [1, 11]]
        self.costs = {'A': 1, 'B': 10, 'C': 100, 'D': 1000}

    def getPossibleMoves(self, playerPos, c, idx, cost):
        coord = playerPos[c][idx]
        otherFigs = []
        for color in playerPos.keys():
            for i in playerPos[color]:
                if i != idx and color != c:
                    otherFigs.append(i)

        # Todo: check if in home:
        if list(coord) == self.homes[c][1]:
            return cost

        if list(coord) in self.homes[c] and list(playerPos[c][0 if idx else 1]) in self.homes[c]:
            return cost
        #
        if coord[0] == 3:
            # have to check whether field above is free
            if (coord[0]-1, coord[1]) in otherFigs:
                return

        otherFigs_target = [[x, y] for x, y in otherFigs if x == 1]
        otherFigs_target_y = sorted([y for x, y in otherFigs_target])
        next_left = next(
            (y for y in otherFigs_target_y if (y < coord[1])), -100)
        next_right = next((y for y in reversed(
            otherFigs_target_y) if (y > coord[1])), 100)

        homeEmpty = True
        colorFalse = False
        lowerCorrect = False
        upperEmpty = True
        for color in playerPos.keys():
            for i in playerPos[color]:
                i = list(i)
                if i in self.homes[c]:
                    if color != c:
                        colorFalse = True
                    elif i == self.homes[c][1] and color == c:
                        lowerCorrect = True
                    else:
                        upperEmpty = False
                    homeEmpty = False

        homeReachable = False
        if next_left < self.homes[c][0][1] < next_right:
            homeReachable = True

        if homeEmpty and homeReachable:
            # move into bottom slot
            retVal = []
            p = copy.deepcopy(playerPos)
            p[c][idx] = tuple(self.homes[c][1])
            retVal.append((p, (
                cost + self.costs[c]*(abs(coord[0]-p[c]
                                      [idx][0])+abs(coord[1]-p[c][idx][1]))
            )))
            return retVal

        if homeReachable and lowerCorrect and upperEmpty:
            # move into top slot
            retVal = []
            p = copy.deepcopy(playerPos)
            p[c][idx] = tuple(self.homes[c][0])
            retVal.append((p, (
                cost + self.costs[c]*(abs(coord[0]-p[c]
                                      [idx][0])+abs(coord[1]-p[c][idx][1]))
            )))
            return retVal

        
        new_pos = [i for i in self.movable_pos if next_left < i[1] < next_right]

        retVal = []
        for i in new_pos:
            p = copy.deepcopy(playerPos)
            p[c][idx] = tuple(i)
            retVal.append((p, (
                cost + self.costs[c]*(abs(coord[0]-p[c]
                                      [idx][0])+abs(coord[1]-p[c][idx][1]))
            )))
        return retVal

    def print(self, playerPos):
        temp = copy.deepcopy(self.map)
        for color in playerPos.keys():
            for i in playerPos[color]:
                temp[i[0], i[1]] = color

        for i in range(temp.shape[0]):
            print("".join(temp[i, :]))
        pass

    @staticmethod
    def fromString(string):
        map = np.array([np.array(list(line.ljust(len(string.splitlines()[0]))))
                       for line in rawstring.splitlines()])

        playerPos = dict()
        for i in 'ABCD':
            playerPos[i] = [(x, y) for x, y in zip(
                np.where(map == i)[0], np.where(map == i)[1])]

        for i in [' ']:
            map[map == i] = '#'
        for i in "ABCD":
            map[map == i] = '.'
        return Field(map), playerPos


In [45]:
m,p = Field.fromString(rawstring)

In [46]:
p

{'A': [(3, 5), (3, 7)],
 'B': [(2, 5), (2, 9)],
 'C': [(3, 3), (3, 9)],
 'D': [(2, 3), (2, 7)]}

In [47]:
import heapq

In [48]:
a = dict()

In [49]:
str(tuple(tuple(b) for b in tuple(p.values())))

'(((3, 5), (3, 7)), ((2, 5), (2, 9)), ((3, 3), (3, 9)), ((2, 3), (2, 7)))'

In [50]:
a[(5,4)] = 7

In [51]:
tuple(tuple(b) for b in tuple(p.values())) in a

False

In [52]:
p

{'A': [(3, 5), (3, 7)],
 'B': [(2, 5), (2, 9)],
 'C': [(3, 3), (3, 9)],
 'D': [(2, 3), (2, 7)]}

In [67]:
solution

19867

In [66]:
openConfigs = [(0,0,p,0)]
configs = dict()
solution = None
count = 0
while len(openConfigs) != 0 and count < 100_000_000:
    cConfig = heapq.heappop(openConfigs)
    idx = tuple(tuple(b) for b in tuple(cConfig[2].values()))
    if (count % 1000) == 0:
        print(f"{len(configs)=}")
        print(f"{len(openConfigs)=}")
    if idx in configs:
        if configs[idx] <= cConfig[3]:
            continue
        else:
            configs[idx] = cConfig[3]
    else:
        configs[idx] = cConfig[3]
    count += 1
    int_rets = 0
    tempList = []
    for c in 'ABCD':
        for i in [0,1]:
            temp = m.getPossibleMoves(cConfig[2],c,i,cConfig[3])
            if type(temp) == np.int64 or type(temp) == int:
                int_rets += 1
            elif temp != None:
                for newConfig in temp:
                    tempList.append(newConfig)
    for t in tempList:
        idx = tuple(tuple(b) for b in tuple(t[0].values()))
        if idx in configs:
            if configs[idx] <= t[1]:
                continue
            else:
                heapq.heappush(openConfigs, (8-int_rets,id(t[0]),t[0],t[1]))
        else:
            heapq.heappush(openConfigs, (8-int_rets,id(t[0]),t[0],t[1]))

    if int_rets == 8:
        if solution == None:
            solution = cConfig[3]
            print(solution)
        if cConfig[3] < solution:
            solution = cConfig[3]
            print(solution)

len(configs)=0
len(openConfigs)=0
19867
len(configs)=140
len(openConfigs)=971
len(configs)=333
len(openConfigs)=1474
len(configs)=333
len(openConfigs)=1473
len(configs)=333
len(openConfigs)=1472
len(configs)=333
len(openConfigs)=1471
len(configs)=333
len(openConfigs)=1470
len(configs)=333
len(openConfigs)=1469
len(configs)=333
len(openConfigs)=1468
len(configs)=333
len(openConfigs)=1467
len(configs)=333
len(openConfigs)=1466
len(configs)=333
len(openConfigs)=1465
len(configs)=333
len(openConfigs)=1464
len(configs)=333
len(openConfigs)=1463
len(configs)=333
len(openConfigs)=1462
len(configs)=333
len(openConfigs)=1461
len(configs)=364
len(openConfigs)=520
len(configs)=364
len(openConfigs)=519
len(configs)=364
len(openConfigs)=518
len(configs)=364
len(openConfigs)=517
len(configs)=364
len(openConfigs)=516
len(configs)=364
len(openConfigs)=515
len(configs)=364
len(openConfigs)=514
len(configs)=606
len(openConfigs)=1580
len(configs)=606
len(openConfigs)=1579
len(configs)=606
len(openConfigs

KeyboardInterrupt: 

In [ ]:
solution

38345

In [ ]:
l = deque([1,2,3])

In [ ]:
l.popleft()

1